In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gc
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras import *
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

# Dataset creation
Training dataset is prepared with functions in the [feature engineering notebook](https://www.kaggle.com/mistag/ventilator-feature-engineering), which is based on [Improvement base on Tensor Bidirect LSTM](https://www.kaggle.com/kensit/improvement-base-on-tensor-bidirect-lstm-0-173/notebook) by [Ken Sit](https://www.kaggle.com/kensit).

In [ ]:
def bvar(series):
    series = abs(series).rolling(window=2).apply(np.prod, raw=True)
    return np.sum(series)*((2/np.pi)**(-2))

def add_features(df):
    df['400'] = (df.index.to_series() / 400).astype(int)
    df['8000'] = (df.index.to_series() / 8000).astype(int)
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df["area_max"] = df.groupby("breath_id")["area"].transform("max")
    df['area_w5_prod'] = abs(df.area).rolling(window=5).apply(np.prod, raw=True)

    df["u_in_max"] = df.groupby("breath_id")["u_in"].transform("max")
    df["u_in_sum"] = df.groupby("breath_id")["u_in"].transform("sum")
    df['u_in_bvar'] = df.groupby("breath_id")["u_in"].transform(bvar)
    df["u_in_lag1"] = df.groupby("breath_id")["u_in"].shift(1)
    df["u_in_lag2"] = df.groupby("breath_id")["u_in"].shift(2)
    df["u_in_lag3"] = df.groupby("breath_id")["u_in"].shift(3)

    df["u_in_cumsum"] = df.groupby("breath_id")["u_in"].cumsum()
    df["u_in_cumsum_reverse"] = df["u_in_sum"] - df["u_in_cumsum"]
    
    df["u_out_sum"] = df.groupby("breath_id")["u_out"].transform("sum")
    df["u_out_cumsum"] = df.groupby("breath_id")["u_out"].cumsum()
    df["u_out_cumsum_reverse"] = df["u_out_sum"] - df["u_out_cumsum"]
    
    df["time_passed"] = df.groupby("breath_id")["time_step"].diff()
    
    df['RC'] = df["R"] + df["C"]
    df['R-C']= df.R - df.C
    df["RC_400_mean"] = df.groupby("400")["RC"].transform("mean")
    df['R-C_8000_mean']= df.groupby("8000")['R-C'].transform("mean")
    #df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df.drop(['id','breath_id','u_out','R','C','area',"u_in",'time_step','400','RC','u_in_sum',"R-C",'u_out_sum'],axis=1)
    return df

In [ ]:
train_ori = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
targets = train_ori['pressure'].to_numpy().reshape(-1, 80)
train_ori.drop(labels='pressure', axis=1, inplace=True)
train = add_features(train_ori)
# normalise the dataset
RS = RobustScaler()
train = RS.fit_transform(train)

# Reshape to group 80 timesteps for each breath ID
train = train.reshape(-1, 80, train.shape[-1])

The test set is created below, using the feature engineering function from the above mentioned notebook:

In [ ]:
test_ori = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
test = add_features(test_ori)
test = RS.transform(test)
test = test.reshape(-1, 80, test.shape[-1])

# Model creation
Model parameters are from [Keras model tuning with Optuna](https://www.kaggle.com/mistag/keras-model-tuning-with-optuna). (The "optimal parameters" will not be exactly the same every time the optimization study is run, so the parameters used below might differ from the model tuning notebook).

In [ ]:
# model creation
def create_lstm_model():

    x0 = tf.keras.layers.Input(shape=(train.shape[-2], train.shape[-1]))  

    lstm_layers = 2 # number of LSTM layers
    lstm_units = [320, 229]
    lstm = Bidirectional(keras.layers.LSTM(lstm_units[0], return_sequences=True))(x0)
    for i in range(lstm_layers-1):
        lstm = Bidirectional(keras.layers.LSTM(lstm_units[i+1], return_sequences=True))(lstm)    
    lstm = Dropout(0.01)(lstm)
    lstm = Dense(100, activation='relu')(lstm)
    lstm = Dense(1)(lstm)

    model = keras.Model(inputs=x0, outputs=lstm)
    model.compile(optimizer="adam", loss="mae")
    
    return model

# Training

In [ ]:
# Function to get hardware strategy
def get_hardware_strategy():
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        tf.config.optimizer.set_jit(True)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    return tpu, strategy

tpu, strategy = get_hardware_strategy()

In [ ]:
EPOCH = 350
BATCH_SIZE = 512
NFOLDS = 2

with strategy.scope():
    kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=2021)
    history = []
    test_preds = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        model = create_lstm_model()
        model.compile(optimizer="adam", loss="mae", metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

        scheduler = ExponentialDecay(1e-3, 400*((len(train)*0.8)/BATCH_SIZE), 1e-5)
        lr = LearningRateScheduler(scheduler, verbose=0)

        history.append(model.fit(X_train, y_train, 
                                 validation_data=(X_valid, y_valid), 
                                 epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr]))
        test_pred = model.predict(test).squeeze().reshape(-1, 1).squeeze()
        test_preds.append(test_pred)    
        
        # save model
        #model.save("lstm_model_fold_{}".format(fold))
        
        del X_train, X_valid, y_train, y_valid, model
        gc.collect()

# Submission

In [ ]:
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
submission["pressure"] = sum(test_preds)/2
submission.to_csv('submission.csv', index=False)